# CAMERA Workshop 2019

Lawrence Berkeley National Laboratory - LBNL

* Support material for the tutorial _Image processing for microCT using scikit-image (Part I)_.

This tutorial will introduce how to analyze three dimensional stacked and volumetric
images in Python, mainly using scikit-image. Here we will learn how to:
  * pre-process data using filtering, binarization and segmentation techniques.
  * inspect, count and measure attributes of objects and regions of interest in the data.
  * visualize 3D data.

Please prepare for the tutorial by [installing the pre-requisite software](preparation.md) beforehand.

For more info:
  * [[CAMERA Workshop 2019]](http://microct.lbl.gov/cameratomo2019/)
  * [[scikit-image]](https://scikit-image.org/)


## Importing some starting modules...

... and adding some bells and whistles.

In [ ]:
from skimage import restoration, data, io

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

plt.rcParams['font.family'] = 'monospace'

Reading `cells.tif`.

In [ ]:
cells = io.imread('../data/cells.tif')

plt.figure(figsize=(15, 10))
plt.imshow(cells[32], cmap='gray')

## Using joblib to parallelize code

Joblib is a set of tools to provide lightweight pipelining in Python. In particular:

1. transparent disk-caching of functions and lazy re-evaluation (known as _memoize pattern_)
2. easy simple parallel computing

Joblib is optimized to be fast and robust on large data in particular and has specific optimizations for numpy arrays.

* [[joblib documentation]](https://joblib.readthedocs.io/en/latest/)

Here we implement a function to apply the bilateral filter, the "classic" way.

In [ ]:
def bilateral_classic_loop(data):
    """A function to apply the bilateral filter on 3D data."""
    data_bilateral = np.empty_like(data)
    for plane, image in enumerate(data):
        data_bilateral[plane] = restoration.denoise_bilateral(image,
                                                              multichannel=False)
    return data_bilateral

%time _ = bilateral_classic_loop(cells)

Now we implement this function using `joblib`. 

In [ ]:
from joblib import Parallel, delayed

# when using n_jobs=-2, all CPUs but one are used.

def bilateral_joblib_loop(data):
    data_bilateral = Parallel(n_jobs=-2)(delayed(
        restoration.denoise_bilateral
    )(plane, multichannel=False) for plane in data)

    return data_bilateral

%time _ = bilateral_joblib_loop(cells)

We can use also `joblib.Memory` to cache results from a function.

In [ ]:
from joblib import Memory

memory = Memory(location='../cache', verbose=0)

@memory.cache
def bilateral_joblib_loop(data):
    data_bilateral = Parallel(n_jobs=2)(delayed(
        restoration.denoise_bilateral
    )(plane, multichannel=False) for plane in data)

    return data_bilateral


# 1st execution
%time _ = bilateral_joblib_loop(cells)

# 2nd execution, cached
%time _ = bilateral_joblib_loop(cells)

## Parallelize operations with Dask.array

Dask allows parallel computing in python, coordinating many NumPy arrays arranged into a grid. Useful when dealing with collections of data that do not fit the RAM in your PC.

Let's split `cells` into four chunks, and map `skimage.filters.gaussian` on each block.

In [ ]:
import dask.array as da
from skimage import filters

depth, rows, cols = cells.shape

%time
cells_dask = da.from_array(cells, chunks=(depth // 2, rows // 2, cols // 2), name=False)
cells_dask_gauss = cells_dask.map_overlap(filters.gaussian, depth=15, sigma=3, boundary='none')

Let's see the structure of `cells_dask`. `.visualize()` shows the task graph:

In [ ]:
cells_dask.visualize()

Some commands to get `.visualize()` working:

`$ conda install graphviz python-graphviz`

`$ sudo apt install graphviz` (on Ubuntu)

Now, the structure of `cells_dask_gauss`.

In [ ]:
cells_dask_gauss.visualize()

Now, we use `.compute()` to actually calculate the operations.

In [ ]:
%time _ = cells_dask_gauss.compute()

## Going beyond

[1] Matt Rocklin's "Scikit-Image and Dask Experiment": https://nbviewer.jupyter.org/gist/mrocklin/ec745d6c2a12dddddb125ef460a4da76

[2] skimage's `util.apply_parallel`: https://scikit-image.org/docs/dev/api/skimage.util.html#skimage.util.apply_parallel

[3] Emmanuelle Gouillart's "Processing X-ray tomography images with Python": http://emmanuelle.github.io/segmentation-of-3-d-tomography-images-with-python-and-scikit-image.html

[4] Valentina Staneva's "Scalable Data Analysis": https://github.com/valentina-s/Dask_ImageXD_2018/blob/master/Scalable-Data-Analysis-with-Dask.ipynb